Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers.

In [2]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response


In [ ]:
###########zero-shot##########
discrete_prompt =  "get the sum of odd numbers from a list of integers [1,3,2,4,6,8,11]"
response_discrete = get_completion_falcon(discrete_prompt)
print(response_discrete)

In [ ]:
###########few-shot##########
kwargs= {"max_length": 900}

few_shot_prompt =  """I will provide you with a few examples of converting a list of integers to a number.
Follow the pattern to convert the given list of integers into a number.
Example 1:
list of integer: '[1,3,7,9,10]'
number: '20'

Example 2:
list of integer: '[1,6,5,7,12]'
number: '13'

Now, convert '[2,3,8,7,16] """
response_few_shot = get_completion_falcon(few_shot_prompt, **kwargs)
print(response_few_shot)

In [ ]:
######### chain-of-thought,
# Prompt for solving an algebra problem with step-by-step explanation - how does the LLM perform?
math_problem_prompt = "Explain step-by-step how to solve the following algebra problem: get the sum of odd numbers from a list of integers [1,4,6,7,9,13]?"
kwargs = {"max_length": 1200}
math_problem_solution = get_completion_falcon(math_problem_prompt, **kwargs)
print(math_problem_solution)

In [ ]:
######### few-shot chain-of-thought
# Prompt for explaining the process of solving basic arithmetic problems
kwargs = {"max_length": 2200}
few_shot_COT_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: get the sum of odd numbers from a list of integers: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: get the sum of odd numbers from a list of integers: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: get the sum of odd numbers from a list of integers: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: get the sum of odd numbers from a list of integers: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)